In [1]:
import pandas as pd
import openpyxl
import datetime

# Load the data
df = pd.read_csv('PRG Sales Comparables Table.csv', encoding='latin1')

# Function to convert the date integer string to short date format
def convert_to_short_date(date_str):
    try:
        # Attempt to convert to integer
        date_int = int(date_str)
        
        # Convert to a date (Excel date starting point)
        date_value = datetime.datetime(1900, 1, 1) + datetime.timedelta(days=date_int - 2)
        
        # Format without leading zeros
        return date_value  # Return as datetime object for proper sorting
    except (ValueError, TypeError):
        return None  # or return date_str to keep the original value

# Apply the conversion to the 'Sale Date' column
df['Sale Date'] = df['Sale Date'].apply(convert_to_short_date)

# Sort the DataFrame by 'Sale Date' in descending order
df = df.sort_values(by='Sale Date', ascending=False)

# Function to format the numbers with commas and convert to text
def format_land_size(size_str):
    try:
        # Convert to integer
        size_int = int(size_str)
        # Format the integer with commas and convert to text
        return f"{size_int:,}"  # This will format with commas
    except (ValueError, TypeError):
        return None  # or return size_str to keep the original value

# Apply the formatting to the 'LAND SIZE (SF)' column
df['LAND SIZE (SF)'] = df['LAND SIZE (SF)'].apply(format_land_size)

# Display unique counties and their corresponding indices
unique_counties = df['County'].unique()
print("Select a county by number:")
for index, county in enumerate(unique_counties):
    print(f"{index}: {county}")

# Get user input for county
try:
    user_input = int(input("Enter the number corresponding to the county: "))
    
    if 0 <= user_input < len(unique_counties):
        selected_county = unique_counties[user_input]
        df_filtered_by_county = df[df['County'] == selected_county]
        
        print(f"\nFiltered DataFrame for county: {selected_county}")

        # Display unique uses for the selected county
        unique_uses = df_filtered_by_county['Use'].unique()
        print("\nSelect a use by number:")
        for index, use in enumerate(unique_uses):
            print(f"{index}: {use}")

        # Get user input for use
        try:
            user_input = int(input("Enter the number corresponding to the use: "))
            
            if 0 <= user_input < len(unique_uses):
                selected_use = unique_uses[user_input]
                df = df_filtered_by_county[df_filtered_by_county['Use'] == selected_use]
                
                print(f"\nFiltered DataFrame for use: {selected_use}")
            else:
                print("Invalid selection. Please select a number from the list.")
        except ValueError:
            print("Invalid input. Please enter a number.")
    else:
        print("Invalid selection. Please select a number from the list.")
except ValueError:
    print("Invalid input. Please enter a number.")

df = df.head(5)

# Load the Excel file
file_path = 'flex_comp_grid.xlsx'
workbook = openpyxl.load_workbook(file_path)
sheet = workbook.active

# Display the contents of the third column for reference
third_column_data = [row[0] for row in sheet.iter_rows(min_col=3, max_col=3, values_only=True)]
print("Original third column data:", third_column_data)

num_duplicates = df.shape[0]

# Start processing each row
for row in range(1, len(third_column_data) + 1):
    original_cell = sheet.cell(row=row, column=3)

    # Move the existing values from the fourth column to the right for all rows
    for j in range(num_duplicates):
        existing_cell = sheet.cell(row=row, column=4)
        new_cell = sheet.cell(row=row, column=4 + j)
        new_cell.value = existing_cell.value

        if existing_cell.has_style:
            new_cell.font = existing_cell.font.copy()
            new_cell.fill = existing_cell.fill.copy()
            new_cell.border = existing_cell.border.copy()
            new_cell.alignment = existing_cell.alignment.copy()
            new_cell.number_format = existing_cell.number_format

    # Duplicating values in the third column
    if row == 4:
        for i in range(num_duplicates):
            new_cell = sheet.cell(row=row, column=3 + i)
            new_cell.value = original_cell.value + i
            if original_cell.has_style:
                new_cell.font = original_cell.font.copy()
                new_cell.fill = original_cell.fill.copy()
                new_cell.border = original_cell.border.copy()
                new_cell.alignment = original_cell.alignment.copy()
                new_cell.number_format = original_cell.number_format
    else:
        for i in range(num_duplicates):
            new_cell = sheet.cell(row=row, column=3 + i)
            new_cell.value = original_cell.value
            if original_cell.has_style:
                new_cell.font = original_cell.font.copy()
                new_cell.fill = original_cell.fill.copy()
                new_cell.border = original_cell.border.copy()
                new_cell.alignment = original_cell.alignment.copy()
                new_cell.number_format = original_cell.number_format

# Insert values from the DataFrame into the specified rows in the Excel sheet
for i in range(len(df)):
    for j, col_index in enumerate([12, 17, 23, 22, 1, 26, 27, 24]):
        cell_to_insert = sheet.cell(row=5 + j, column=3 + i)
        cell_to_insert.value = df.iloc[i, col_index]

# Save the modified workbook to a new Excel file
output_file_path = 'duplicated_columns_with_incremented_fourth_row.xlsx'
workbook.save(output_file_path)

print(f"Duplicated columns with incremented fourth row saved to '{output_file_path}'.")


Select a county by number:
0: Frederick
1: Carroll
2: Washington
3: Allegany
4: Prince George
5: Garrett
6: Washington 
7: Frederick 
8: Montgomery
9: Saint Mary's
10: Charles
11: Jefferson
12: Calvert
13: Carroll 
14: Baltimore
15: Anne Arundel
16: Howard


Enter the number corresponding to the county:  0



Filtered DataFrame for county: Frederick

Select a use by number:
0: Warehouse/ Flex
1: Vacant Land
2: Office Condo
3: Professional Office
4: Retail
5: Medical Office
6: Warehouse
7: Mixed
8: Apartment
9: Church
10: Restaurant
11: Daycare
12: Convenience Store
13: Auto Sales/ Service
14: Special Purpose
15: Mixed 
16: Warehouse Condo


Enter the number corresponding to the use:  0



Filtered DataFrame for use: Warehouse/ Flex
Original third column data: [None, None, None, 1, None, None, None, None, None, '=C7/C9', None, None, None, None, '=C6/C7', None, 'FEE SIMPLE', 0, '=C15*C18+C15', 'TYPICAL', 0, '=C19*C21+C19', 'ARMS-LENGTH', 0, '=C22*C24+C22', 'YES', 0.08, '=C25*C27+C25', 'STABLE', 0, '=C28*C30+C28', None, 0, 0, 0, 0, 0, 0, 0, '=SUM(C33:C39)', '=C31*C40+C31']
Duplicated columns with incremented fourth row saved to 'duplicated_columns_with_incremented_fourth_row.xlsx'.


C:\Users\micha\AppData\Local\Temp\ipykernel_308\2739832512.py:105: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  new_cell.font = existing_cell.font.copy()
C:\Users\micha\AppData\Local\Temp\ipykernel_308\2739832512.py:106: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  new_cell.fill = existing_cell.fill.copy()
C:\Users\micha\AppData\Local\Temp\ipykernel_308\2739832512.py:107: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  new_cell.border = existing_cell.border.copy()
C:\Users\micha\AppData\Local\Temp\ipykernel_308\2739832512.py:108: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or cell.obj = cell.obj + other).
  new_cell.alignment = existing_cell.alignment.copy()
C:\Users\micha\AppData\Local\Temp\ipykernel_308\2739832512.py:127: DeprecationWarning: Call to deprecated function copy (Use copy(obj) or 

In [70]:
df.columns

Index(['PRG \nReference Number', 'County', 'Use', 'Address', 'City', 'State',
       'TAX ID ', 'Tax ID ', 'Map/ Grid/ Parcel', 'Legal Desc.', 'Grantor',
       'Grantee', 'Sale Date', 'Deed', 'Property Rights', 'Conditions at Sale',
       'Time on Market', 'Sale Price', 'Price/GBA', 'Vacancy', 'NOI',
       'CAP RATE', 'LAND SIZE (SF)', 'GBA (SF)', 'YEAR BUILT', 'Parking',
       'Zone', 'Utilities', 'APT ONLY\nNumber of Units',
       'APT ONLY\nNumber of Rooms', 'APT ONLY\nPrice per Unit',
       'APT ONLY\nPrice per Room', 'Comments', 'Unnamed: 33'],
      dtype='object')

In [86]:
import pandas as pd

# Sample DataFrame with some numeric values as strings
data = {'LAND SIZE (SF)': ['1234', '56789', '101112', '3000', '45000']}
df = pd.DataFrame(data)

# Function to format the numbers with commas and convert to text
def format_land_size(size_str):
    try:
        # Convert to integer
        size_int = int(size_str)
        # Format the integer with commas and convert to text
        return f"{size_int:,}"  # This will format with commas
    except (ValueError, TypeError):
        return None  # or return size_str to keep the original value

# Apply the formatting to the 'LAND SIZE (SF)' column
df['LAND SIZE (SF)'] = df['LAND SIZE (SF)'].apply(format_land_size)

print(df)


  LAND SIZE (SF)
0          1,234
1         56,789
2        101,112
3          3,000
4         45,000


In [95]:
df['Sale Date'].head()

334    9/19/2022
159    9/17/2018
386     8/4/2022
294    8/24/2018
80     8/22/2019
Name: Sale Date, dtype: object